<a href="https://colab.research.google.com/github/littlejacinthe/torchaudio/blob/main/Neuralnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Tutorial by Valerio Velardo
---



In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [9]:
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 0.001

In [10]:
class FeedForwardNet(nn.Module):

  def __init__(self):
    super().__init__() # nn.Module functions

    self.flatten = nn.Flatten()
    self.dense_layers = nn.Sequential( #trick to get several layers into one component
        nn.Linear(28*28, 256), #images in the dataset are of size 28x28 --> Flattened
        nn.ReLU(), #activation layer
        nn.Linear(256, 10) #output layer
    )
    self.softmax = nn.Softmax(dim=1)

  def forward(self, input_data): #how to manipulate the data
    flattened_data = self.flatten(input_data)
    logits = self.dense_layers(flattened_data)
    predictions = self.softmax(logits)
    return predictions

In [11]:
def download_mnist_datasets():

  train_data = datasets.MNIST(
      root="data",
      download=True,
      train=True,
      transform=ToTensor() # normalized btw 0 and 1
  )

  validation_data = datasets.MNIST(
      root="data",
      download=True,
      train=False,
      transform=ToTensor() # normalized btw 0 and 1
  )
  
  return train_data, validation_data

In [12]:
def train_one_epoch(model, data_loader, loss_fn, optimiser, device):
  
  for inputs, targets in data_loader:
    inputs, targets = inputs.to(device), targets.to(device)

    #calculate loss
    predictions = model(inputs)
    loss = loss_fn(predictions, targets)

    #backpropagate loss and update weights
    optimiser.zero_grad() # the optimiser calculates gradients at each iteration to decide how to update the weights, this resets the gradients to 0
    loss.backward()
    optimiser.step()


  print(f"Loss: {loss.item()}") #prints the loss for the last batch 


def train(model, data_loader, loss_fn, optimiser, device, epochs):
  
  for i in range(epochs):
    print(f"Epoch {i+1}")
    train_one_epoch(model, data_loader, loss_fn, optimiser, device)
    print("------------------------")

  print("Training done")

In [13]:
from torch.nn.modules.loss import CrossEntropyLoss
if __name__ == "__main__":

  # download dataset
  train_data, _ = download_mnist_datasets()
  print("MNIST dataset downloaded")

  # create dataloader for dataset
  train_data_loader = DataLoader(train_data, batch_size=BATCH_SIZE)

  #build model
  if torch.cuda.is_available():
    device = "cuda"
  else:
    device = "cpu"
  print(f"Using {device} device")

  feed_forward_net = FeedForwardNet().to(device)

  #instantiate loss function & optimiser
  loss_fn = nn.CrossEntropyLoss()
  optimiser = torch.optim.Adam(feed_forward_net.parameters(), lr=LEARNING_RATE)

  #train model
  train(feed_forward_net, train_data_loader, loss_fn, optimiser, device, EPOCHS)

  #store the model
  torch.save(feed_forward_net.state_dict(), "feedforwardnet.pth")
  print("Model trained and stored at feedforwardnet.pth")

MNIST dataset downloaded
Using cpu device
Epoch 1
Loss: 1.5094488859176636
------------------------
Epoch 2
Loss: 1.4943381547927856
------------------------
Epoch 3
Loss: 1.4868143796920776
------------------------
Epoch 4
Loss: 1.4790884256362915
------------------------
Epoch 5
Loss: 1.4740031957626343
------------------------
Epoch 6
Loss: 1.4727061986923218
------------------------
Epoch 7
Loss: 1.4728583097457886
------------------------
Epoch 8
Loss: 1.4728723764419556
------------------------
Epoch 9
Loss: 1.4723901748657227
------------------------
Epoch 10
Loss: 1.4729056358337402
------------------------
Training done
Model trained and stored at feedforwardnet.pth
